## Inventory and Pricing Analysis Report(SQL)

In [1]:
import pandas as pd
import sqlite3


In [2]:
# Read CSV files into DataFrames
inv_item = pd.read_csv('inv_item_exported.csv')
inv_item_price = pd.read_csv('inv_item_price_exported.csv')
inv_item_stock = pd.read_csv('inv_item_stock_exported.csv')
inv_stockroom = pd.read_csv('inv_stockroom_exported.csv')

# Creact SQLite database 
conn = sqlite3.connect('inventory.db')

# Write DataFrames to SQLite tables
inv_item.to_sql('inv_item', conn, if_exists='replace', index=False)
inv_item_price.to_sql('inv_item_price', conn, if_exists='replace', index=False)
inv_item_stock.to_sql('inv_item_stock', conn, if_exists='replace', index=False)
inv_stockroom.to_sql('inv_stockroom', conn, if_exists='replace', index=False)

5

# Why SQLite3?

In this project, we chose to use SQLite3 for several reasons:

1. **Lightweight and Easy to Use**: SQLite3 is a lightweight database engine that is easy to set up and use. It doesn't require a separate server process, making it ideal for small to medium-sized projects and for use in development environments.

2. **Integration with Python**: SQLite3 has excellent integration with Python through the `sqlite3` module, which is included in the Python standard library. This makes it straightforward to create, query, and manage SQLite databases directly from Python code.

3. **Portability**: SQLite databases are stored in a single file, making them highly portable. This is useful for sharing the database with others or moving it between different environments.

4. **Performance**: For many use cases, SQLite3 provides sufficient performance. It is optimized for read-heavy operations and can handle moderate write loads efficiently.

5. **SQL Support**: SQLite3 supports most of the SQL standard, allowing us to use familiar SQL queries to interact with the database. This makes it easy to perform complex queries and data manipulations.

6. **No Configuration Required**: Unlike other database systems, SQLite3 requires no configuration or administration. This reduces the overhead of managing the database and allows us to focus on the data analysis tasks.

Overall, SQLite3 provides a simple, efficient, and effective solution for managing and querying the inventory data in this project.

In [3]:
# 1. Average price of items with physical inventory and those without
query1 = """
SELECT 
    CASE WHEN has_physical_inventory = 1 THEN 'With Inventory' ELSE 'Without Inventory' END AS inventory_status,
    AVG(buying_price) AS average_price
FROM inv_item
WHERE buying_price IS NOT NULL
GROUP BY inventory_status;

"""
result1 = pd.read_sql_query(query1, conn)

print(result1)

    inventory_status  average_price
0     With Inventory      277.46309
1  Without Inventory       24.00000


The output of the query provides insights into the average buying price of items based on their inventory status.By understanding the price differences and inventory status, the marketing team can tailor their strategies to maximize sales and manage inventory more effectively.

In [4]:

# 2. Show which storeroom has the most inventory
query2 = """
SELECT stockroom_id, SUM(quantity) AS total_quantity
FROM inv_item_stock
GROUP BY stockroom_id
ORDER BY total_quantity DESC
LIMIT 1
"""
result2 = pd.read_sql_query(query2, conn)
print(result2)



   stockroom_id  total_quantity
0           342           90669


By prioritizing the stockroom with the highest inventory levels, the marketing team can optimize their promotional strategies and achieve more efficient inventory management. This approach ensures that high-demand items are readily available, reducing the risk of stockouts and enhancing overall customer satisfaction.


In [5]:
# 3. Time in minutes since item prices were last modified
query3 = """
SELECT item_id, price,
       (strftime('%s', 'now') - strftime('%s', date_changed)) / 60 AS minutes_since_modified
FROM inv_item_price;
"""
result3 = pd.read_sql_query(query3, conn)
print(result3)



      item_id price  minutes_since_modified
0        90.0   OPD                28746107
1        30.0   OPD                28746107
2       635.0   OPD                28746107
3       880.0   OPD                28746107
4       515.0   OPD                28746107
...       ...   ...                     ...
1061   1105.0  None                28746107
1062      7.0  None                28746107
1063    120.0  None                28746107
1064    110.0  None                28746107
1065      1.5  None                28746107

[1066 rows x 3 columns]


By addressing these points, the marketing team can ensure that their pricing strategies are up-to-date and effective, which can lead to improved sales and customer satisfaction.

In [6]:

# 4. List items that are running out of stock
query4 = """
SELECT * FROM inv_item_stock
WHERE quantity < 5
"""
result4 = pd.read_sql_query(query4, conn)
print(result4)


     item_stock_id  stockroom_id  item_id  quantity  \
0               10           343     1587         4   
1               33           343     1579         2   
2               41           343     1577         4   
3               46           343     1508         4   
4               51           343     1543         0   
..             ...           ...      ...       ...   
105          34469           344     1582      -100   
106          34493           342     1296         1   
107          34500           346     1130       -21   
108          34501           346      883       -10   
109          34502           342     1452       -15   

                                     uuid  
0    0954290b-6ec1-40d2-bbc8-19c25d44f178  
1    916501cc-6e2f-4109-97e2-de625d451b4a  
2    0712ac78-2ae2-44c5-b07c-b3fdd194511b  
3    c7b0f721-753a-4a50-83ce-db79b5913831  
4    e468153f-846d-4955-a221-65aa55fa93d8  
..                                    ...  
105  96f7719f-6de8-469a-836c-21

The marketing team can ensure that inventory levels are maintained, discrepancies are resolved, and promotional strategies are effectively implemented to maximize sales and customer satisfaction.

In [7]:
# 5. List items whose quantity is Zero (0) and how long (In minutes) they have been in this state
query5 = """
SELECT
    i.name AS item_name,
    s.quantity AS current_quantity,
    ROUND((julianday('now') - julianday(i.date_changed)) * 24 * 60) AS minutes_since_changed
FROM inv_item AS i
INNER JOIN inv_item_stock AS s ON i.item_id = s.item_id
WHERE s.quantity = 0;

"""
result5 = pd.read_sql_query(query5, conn)
print(result5)

                                    item_name  current_quantity  \
0                                 Cannula G24                 0   
1                          Vitamin B12 1mg/Ml                 0   
2                        Aspirin(cardi)- 80mg                 0   
3  Normal Saline - 500ml 0.9% NA+CL 1/2 liter                 0   

   minutes_since_changed  
0              5104845.0  
1              5107824.0  
2              4950954.0  
3              5104830.0  


The marketing team can ensure that inventory levels are maintained, restocking is prioritized effectively, and promotional strategies are implemented to maximize sales and customer satisfaction.

### In Conclusion

To optimize sales and enhance inventory management, the marketing team should implement a multifaceted strategy. Firstly, promotional efforts should be concentrated on the top 10 high-demand items. By highlighting these products in marketing campaigns, the team can maximize sales and ensure that popular items are readily available to meet customer demand.

In parallel, it is essential to address low-demand items. Offering discounts or clearance sales for the top 10 low-demand items can help clear out inventory that is not moving as quickly. This approach not only frees up valuable storage space but also provides an opportunity to attract price-sensitive customers and increase overall sales volume.

Effective stock management is another critical component. The stockroom with the most inventory should be meticulously managed to ensure it is well-stocked with high-demand items. This proactive approach helps prevent stockouts, ensuring that customers can always find the products they are looking for, thereby enhancing customer satisfaction and loyalty.

Lastly, a thorough analysis of the impact of pricing on sales is crucial. By understanding how different price points affect sales volume, the marketing team can make informed decisions to adjust prices strategically. This data-driven approach allows for the optimization of pricing strategies to maximize revenue, ensuring that the healthcare platform remains competitive and profitable.

By integrating these strategies, the marketing team can create a balanced approach that not only drives sales but also maintains efficient inventory levels, ultimately contributing to the overall success of the healthcare platform.
